In [ ]:
# accessing sqlite database with python

In [8]:
import sqlite3
import pandas as pd


In [7]:
# open connection to database
con = sqlite3.connect('chinook.db') # chinook is in the same folder as this file
# for other SQL databases, you would need a different driver
# also, you would need to install the driver
# most likely you would need a username and password as well

# con = sqlite3.connect('chinookbad.db') 
# con = sqlite3.connect('chinook_fresh.db') # will create a brand new database if it does not exist
# get status
print(con)
# con.close() # you have to close the database connection if you want to perform any changes

In [9]:
# load tracks table into a dataframe
df = pd.read_sql_query("select * from tracks", con)
df.head()
# now you can do the usual pandas stuff

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [10]:
# load view into a dataframe
rock_df = pd.read_sql_query("select * from v_rock_songs", con)
rock_df.head()

,TrackName,AlbumTitle,ArtistName,Genre,MediaType,time_ms,seconds,minutes,UnitPrice
0,Dazed And Confused,The Song Remains The Same (Disc 1),Led Zeppelin,Rock,MPEG audio file,1612329,1612,26,0.99
1,Space Truckin',The Final Concerts (Disc 2),Deep Purple,Rock,MPEG audio file,1196094,1196,19,0.99
2,Dazed And Confused,BBC Sessions [Disc 2] [Live],Led Zeppelin,Rock,MPEG audio file,1116734,1116,18,0.99
3,We've Got To Get Together/Jingo,Santana Live,Santana,Rock,MPEG audio file,1070027,1070,17,0.99
4,Funky Piano,Santana Live,Santana,Rock,MPEG audio file,934791,934,15,0.99


In [11]:
rock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1297 entries, 0 to 1296
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TrackName   1297 non-null   object 
 1   AlbumTitle  1297 non-null   object 
 2   ArtistName  1297 non-null   object 
 3   Genre       1297 non-null   object 
 4   MediaType   1297 non-null   object 
 5   time_ms     1297 non-null   int64  
 6   seconds     1297 non-null   int64  
 7   minutes     1297 non-null   int64  
 8   UnitPrice   1297 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 91.3+ KB


In [13]:
# you can do filtering in pandas
# you can do filtering in sql

# let's filter view by minutes
rock_df[rock_df['minutes'] > 5].tail()

,TrackName,AlbumTitle,ArtistName,Genre,MediaType,time_ms,seconds,minutes,UnitPrice
186,Sin,Core,Stone Temple Pilots,Rock,MPEG audio file,364800,364,6,0.99
187,Black Sabbath,Speak of the Devil,Ozzy Osbourne,Rock,Protected AAC audio file,364180,364,6,0.99
188,How Many Say I,Van Halen III,Van Halen,Rock,MPEG audio file,363937,363,6,0.99
189,No Sign of Yesterday,The Best Of Men At Work,Men At Work,Rock,MPEG audio file,362004,362,6,0.99
190,Hands All Over,A-Sides,Soundgarden,Rock,MPEG audio file,362475,362,6,0.99


In [15]:
# let's do the same query in SQL
long_rock_songs = pd.read_sql_query("select * from v_rock_songs where minutes > 5", con)
# if you have a huge database you might want to filter earlier in SQL
# if you have a small database you might want to filter later in pandas
long_rock_songs.tail()

,TrackName,AlbumTitle,ArtistName,Genre,MediaType,time_ms,seconds,minutes,UnitPrice
186,Sin,Core,Stone Temple Pilots,Rock,MPEG audio file,364800,364,6,0.99
187,Black Sabbath,Speak of the Devil,Ozzy Osbourne,Rock,Protected AAC audio file,364180,364,6,0.99
188,How Many Say I,Van Halen III,Van Halen,Rock,MPEG audio file,363937,363,6,0.99
189,No Sign of Yesterday,The Best Of Men At Work,Men At Work,Rock,MPEG audio file,362004,362,6,0.99
190,Hands All Over,A-Sides,Soundgarden,Rock,MPEG audio file,362475,362,6,0.99


In [16]:
# we can do queries without pandas

# get cursor
cur = con.cursor() # you can use this cursor to execute queries
# get status
print(cur)

In [17]:
# get all tables
cur.execute("select name from sqlite_master where type = 'table'")
# this is a special query that returns all tables in the database
# you can also use this query to get all views
# show all tables
table_list = cur.fetchall() # this will return a list of tuples
table_list
# why list of tuples?
# TODO: find out why list of tuples NOT list of strings

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',),
 ('results',)]

In [18]:
# lets get all views in our database
view_list = cur.execute("select name from sqlite_master where type = 'view'").fetchall()
view_list

[('v_top_playlist',),
 ('v_tracks',),
 ('v_long_songs',),
 ('v_full_tracks',),
 ('v_full_tracks_2',),
 ('v_rock_songs',)]

In [23]:
# be careful with fetchall() for large databases
# you can use fetchone() to get one row at a time
# how to fetch rows one at a time

# get all genres
result = cur.execute("select * from genres")
# get results one at a time
# result.fetchone()
# result.fetchone()
# we could use a loop to get all results
for row in result: # think of result in a way similar to how range works, it is an iterable
    print(row)
# you can also use fetchmany() to get a number of rows at a time
# in this case it will give us nothing because just like with file reading
# we have to reset the cursor to the beginning of the result
# in this case we would do this by executing the query again
result = cur.execute("select * from genres")
result.fetchmany(5)

(1, 'Rock')
(2, 'Jazz')
(3, 'Metal')
(4, 'Alternative & Punk')
(5, 'Rock And Roll')
(6, 'Blues')
(7, 'Latin')
(8, 'Reggae')
(9, 'Pop')
(10, 'Soundtrack')
(11, 'Bossa Nova')
(12, 'Easy Listening')
(13, 'Heavy Metal')
(14, 'R&B/Soul')
(15, 'Electronica/Dance')
(16, 'World')
(17, 'Hip Hop/Rap')
(18, 'Science Fiction')
(19, 'TV Shows')
(20, 'Sci Fi & Fantasy')
(21, 'Drama')
(22, 'Comedy')
(23, 'Alternative')
(24, 'Classical')
(25, 'Opera')


[(1, 'Rock'),
 (2, 'Jazz'),
 (3, 'Metal'),
 (4, 'Alternative & Punk'),
 (5, 'Rock And Roll')]

In [24]:
# lets get customers from USA
result = cur.execute("select * from customers where country = 'USA'")
us_customers_list = result.fetchall()
us_customers_list # it is up to us to make sense of this data
# these are tuples without column names

[(16,
  'Frank',
  'Harris',
  'Google Inc.',
  '1600 Amphitheatre Parkway',
  'Mountain View',
  'CA',
  'USA',
  '94043-1351',
  '+1 (650) 253-0000',
  '+1 (650) 253-0000',
  'fharris@google.com',
  4),
 (17,
  'Jack',
  'Smith',
  'Microsoft Corporation',
  '1 Microsoft Way',
  'Redmond',
  'WA',
  'USA',
  '98052-8300',
  '+1 (425) 882-8080',
  '+1 (425) 882-8081',
  'jacksmith@microsoft.com',
  5),
 (18,
  'Michelle',
  'Brooks',
  None,
  '627 Broadway',
  'New York',
  'NY',
  'USA',
  '10012-2612',
  '+1 (212) 221-3546',
  '+1 (212) 221-4679',
  'michelleb@aol.com',
  3),
 (19,
  'Tim',
  'Goyer',
  'Apple Inc.',
  '1 Infinite Loop',
  'Cupertino',
  'CA',
  'USA',
  '95014',
  '+1 (408) 996-1010',
  '+1 (408) 996-1011',
  'tgoyer@apple.com',
  3),
 (20,
  'Dan',
  'Miller',
  None,
  '541 Del Medio Avenue',
  'Mountain View',
  'CA',
  'USA',
  '94040-111',
  '+1 (650) 644-3358',
  None,
  'dmiller@comcast.com',
  4),
 (21,
  'Kathy',
  'Chase',
  None,
  '801 W 4th Street',
 

In [26]:
# get column names from customers
# we can use the description attribute of the cursor
# this will give us a list of tuples
# each tuple will have 5 elements
# 0 - column name
# 1 - type code
# 2 - display size
# 3 - internal size
# 4 - precision

# get column names
from pyparsing import col


cur.description # so lot of extra information but we only ned the first element
# we can use list comprehension to get the first element of each tuple

column_names = [tup[0] for tup in cur.description]
column_names

['CustomerId',
 'FirstName',
 'LastName',
 'Company',
 'Address',
 'City',
 'State',
 'Country',
 'PostalCode',
 'Phone',
 'Fax',
 'Email',
 'SupportRepId']

In [7]:
# one typical approach would be to create a customer class
# and then create a list of customer objects
# we can use the zip function to create a list of tuples

class Customer:
    # constructor
    def __init__(self, id, first_name, last_name, company, address, city, state, country, postal_code, phone, fax, email, support_rep_id):
        self.id = id
        self.first_name = first_name
        self.last_name = last_name
        self.company = company
        self.address = address
        self.city = city
        self.state = state
        self.country = country
        self.postal_code = postal_code
        self.phone = phone
        self.fax = fax
        self.email = email
        self.support_rep_id = support_rep_id 
    
    # for printing
    def __repr__(self):
        return f"{self.first_name} {self.last_name} {self.email} {self.country}"

In [29]:
us_customers_list[0]

(16,
 'Frank',
 'Harris',
 'Google Inc.',
 '1600 Amphitheatre Parkway',
 'Mountain View',
 'CA',
 'USA',
 '94043-1351',
 '+1 (650) 253-0000',
 '+1 (650) 253-0000',
 'fharris@google.com',
 4)

In [31]:
# create a list of customer objects
us_customers = [Customer(*tup) for tup in us_customers_list]
# this works because *tup will give me the elements of the tuple as individual arguments
# this is called unpacking

# print first 3 customers
us_customers[:3]

[Frank Harris fharris@google.com USA,
 Jack Smith jacksmith@microsoft.com USA,
 Michelle Brooks michelleb@aol.com USA]

In [32]:
first_customer = us_customers[0]
first_customer.company, first_customer.phone

('Google Inc.', '+1 (650) 253-0000')

In [33]:
# using data classes for storing rows is very typical
# you can also use named tuples - they are immutable
# you can also use dictionaries - they are mutable

# there is an alternative way to do this
# we can use the sqlite3.Row class
# this will allow us to access columns by name

# get cursor
cur = con.cursor()
# set row factory
cur.row_factory = sqlite3.Row
# get all customers from USA
result = cur.execute("select * from customers where country = 'USA'")
# get first row
row = result.fetchone()
# get column names
column_names = row.keys()
# get column values
column_values = [row[col] for col in column_names]
# create a dictionary
row_dict = dict(zip(column_names, column_values))
row_dict


{'CustomerId': 16,
 'FirstName': 'Frank',
 'LastName': 'Harris',
 'Company': 'Google Inc.',
 'Address': '1600 Amphitheatre Parkway',
 'City': 'Mountain View',
 'State': 'CA',
 'Country': 'USA',
 'PostalCode': '94043-1351',
 'Phone': '+1 (650) 253-0000',
 'Fax': '+1 (650) 253-0000',
 'Email': 'fharris@google.com',
 'SupportRepId': 4}

In [ ]:
# CRUD - means create, read, update, delete
# we can do CRUD in SQL
# in SQL - select, insert, update, delete
# so select is read, insert is create, update is update, delete is delete

In [ ]:
# next we will look at create, update and delete of data in SQL

In [1]:
# lets get a connection and cursor again
import sqlite3
con = sqlite3.connect('chinook.db')
cur = con.cursor()

In [4]:
# let's insert a new artist in our database
# we will use the execute method of the cursor
# we will use the commit method of the connection

# documentation for SQL insert at SQLite tutorial
# https://www.sqlitetutorial.net/sqlite-insert/

# insert artist
cur.execute("insert into artists (name) values ('Jumprava')")
# commit changes
con.commit()

In [6]:
# how to do parameterized queries
# we can use the ? placeholder

# insert artist

# you use parameterized queries to avoid SQL injection
# SQL injection is a big security risk
# XKCD comic on SQL injection
# https://xkcd.com/327/

# you use parameterized when you do not know the safety of the data
band_name = input("Enter band name: ") # attacker could enter '; drop table artists; --
# you might have been tempted to use f-strings
# but f-strings are not safe for SQL queries when combined with user input

cur.execute("insert into artists (name) values (?)", (band_name,))
con.commit()

In [9]:
# let's create a new customer
# we will use the execute method of the cursor

new_customer = Customer(None, 'John', 'Doe', "Trump Plaza", '123 Main St', 'New York', 'NY', 'USA', '10001', '555-555-5555', None, '', None)

In [11]:
# let's insert a new customer

# insert customer

# we can use the lastrowid attribute of the cursor to get the id of the last inserted row

# safe way of inserting data
cur.execute("insert into customers (FirstName, LastName,\
     company, address, city, state, country, PostalCode, phone, fax, email, supportRepId)\
         values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
        (   new_customer.first_name, 
            new_customer.last_name, 
            new_customer.company,
            new_customer.address, 
            new_customer.city, 
            new_customer.state, 
            new_customer.country, 
            new_customer.postal_code, 
            new_customer.phone, 
            new_customer.fax, 
            new_customer.email, 
            new_customer.support_rep_id))
# you could insert more customers here

# once you are done inserting customers
con.commit()

In [12]:
# let's update a customer

# update customer

# let's update the customer we just inserted

# we will use the execute method of the cursor

# we will use the commit method of the connection

# documentation for SQL update at SQLite tutorial
# https://www.sqlitetutorial.net/sqlite-update/

# update customer   

# we will find ALL!! rows where city is New York and will rename them to New York City
cur.execute("update customers set city = 'New York City' where city = 'New York'")

# once you are done updating customers
con.commit()

# so if you want to only change one row, the best way is to use the rowid
# you would need to know the rowid of the row you want to update


In [13]:
# parameterized queries are safer
# use them to update a single row

new_city_name = input("Enter new city name: ") # someone could enter New York City'; drop table customers; --

# update documentation at SQLIte tutorial
# https://www.sqlitetutorial.net/sqlite-update/

# safe way to update data
cur.execute("update customers set city = ? where city = 'New York City'", (new_city_name,))

# once you are done updating customers
con.commit()

In [2]:
# let's update customer
# UPDATE employees
# SET city = 'Toronto',
#     state = 'ON',
#     postalcode = 'M5P 2N7'
# WHERE
#     employeeid = 4;

# if these values are not safe this is how we would do it
# we would use parameterized queries
cur.execute("update employees set city = ?, state = ?, postalcode = ? where employeeid = ?", 
    ('Toronto', 'ON', 'M5P 2N7', 4))

# once you are done updating customers
con.commit()


In [14]:
# finally delete a customer

# delete customer

# let's delete the customer we just inserted

# we will use the execute method of the cursor

# we will use the commit method of the connection

# delete last customer

# documentation for SQL delete at SQLite tutorial
# https://www.sqlitetutorial.net/sqlite-delete/

# delete will delete all rows that match the condition!!!

# delete last customer

# we will delete the last customer we inserted

# using subquery to obtain the id of the last customer
cur.execute("delete from customers where customerId = (select max(customerId) from customers)")

con.commit()

# turns out not all sqlite versions have extended delete
# https://www.sqlite.org/lang_delete.html



In [5]:
# how to delete multiple rows

# delete all customers from USA
# let's delete using parameterized queries
# alternative way to delete last customer from USA
# cur.execute("DELETE FROM customers WHERE country = ? ORDER BY CustomerId DESC LIMIT 1", ('USA',))
# cur.execute("DELETE FROM customers ORDER BY CustomerId DESC LIMIT 1")
# con.commit()

OperationalError: near "ORDER": syntax error

In [6]:
# check sqlite version
# sqlite3.version
sqlite3.version # this is library version

'2.6.0'

In [7]:
# check version of sqlite built into Python
sqlite3.sqlite_version # this is the version of sqlite database built into Python

'3.37.2'

In [11]:
# CREATE TABLE manager (
# 	contact_id INTEGER PRIMARY KEY,
# 	first_name TEXT NOT NULL,
# 	last_name TEXT NOT NULL,
# 	email TEXT NOT NULL UNIQUE,
# 	phone TEXT NOT NULL UNIQUE
# );

# let's create a table

# create table documentation at SQLite tutorial
# https://www.sqlitetutorial.net/sqlite-create-table/

# create table

# i am using """ to create a multi-line string
# by saying not null i am saying that the column is required
# by saying unique i am saying that the column must have unique values
# by requiring unique values in effect i am saying this column could have been a primary key
cur.execute("""CREATE TABLE managers 
		(contact_id integer PRIMARY KEY, 
		artist_id integer not null,
		first_name text not null, 
		last_name text not null, 
		email text not null unique, 
		phone text not null unique)""")

# once you are done creating the table

con.commit()



In [14]:
# TODO add foreign key constraint
# https://www.sqlitetutorial.net/sqlite-foreign-key/
# FOREIGN KEY(trackartist) REFERENCES artist(artistid)

# add foreign key constraint for managers table

# cur.execute("ALTER TABLE managers ADD FOREIGN KEY(artist_id) REFERENCES artists(ArtistId)")

# con.commit()

# FIXME add foreign key constraint for managers table


OperationalError: near "FOREIGN": syntax error